# GameLens AI - Phase 1 ROAS Forecasting

This notebook demonstrates the complete Phase 1 pipeline for GameLens AI:
1. Data loading and validation
2. Feature engineering from early-day data (D1-D3)
3. ROAS forecasting model training
4. Generating actionable recommendations

## Overview
The goal is to predict D15/D30/D45/D90 ROAS from just the first 3 days of campaign data, enabling faster go/no-go decisions on UA campaigns.

In [1]:
# Import required libraries
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from utils.data_loader import GameLensDataLoader
from utils.feature_engineering import GameLensFeatureEngineer
from utils.roas_forecaster import GameLensROASForecaster

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## 1. Data Loading and Validation

Load Unity Ads data from Android and iOS platforms and validate data quality.

In [2]:
# Initialize data loader
data_loader = GameLensDataLoader()

# Load all data
print("📊 Loading Unity Ads data...")
platform_data = data_loader.load_all_data()

# Display what we loaded
for platform, data_types in platform_data.items():
    print(f"\n{platform}:")
    for data_type, df in data_types.items():
        print(f"  - {data_type}: {df.shape}")

# Combine platforms
print("\n🔄 Combining platform data...")
combined_data = data_loader.combine_platforms(platform_data)

# Display combined data
print("\nCombined datasets:")
for data_type, df in combined_data.items():
    print(f"  - {data_type}: {df.shape}")
    if not df.empty:
        print(f"    Columns: {list(df.columns)}")
        print(f"    Sample data:")
        print(df.head(3))
        print()

INFO:utils.data_loader:Using data directory: /Users/uzairzafar/Desktop/codebase/gamlens_analytics/Campaign Data
INFO:utils.data_loader:Loaded adspend_revenue for Android: (191, 6)
INFO:utils.data_loader:Loaded level_progression for Android: (18, 24)
INFO:utils.data_loader:Loaded retention for Android: (15, 14)
INFO:utils.data_loader:Loaded roas for Android: (15, 15)
INFO:utils.data_loader:Loaded adspend_revenue for iOS: (105, 6)
INFO:utils.data_loader:Loaded level_progression for iOS: (5, 24)
INFO:utils.data_loader:Loaded retention for iOS: (4, 14)
INFO:utils.data_loader:Loaded roas for iOS: (4, 15)


📊 Loading Unity Ads data...

Android:
  - adspend_revenue: (191, 6)
  - level_progression: (18, 24)
  - retention: (15, 14)
  - roas: (15, 15)

iOS:
  - adspend_revenue: (105, 6)
  - level_progression: (5, 24)
  - retention: (4, 14)
  - roas: (4, 15)

🔄 Combining platform data...

Combined datasets:
  - adspend_revenue: (296, 7)
    Columns: ['country', 'day', 'installs', 'cost', 'ad_revenue', 'revenue', 'platform']
    Sample data:
  country        day  installs    cost  ad_revenue  revenue platform
0  France 2025-08-01         1  0.0500      1.4756      0.0  Android
1   India 2025-08-01         6  0.1898      0.0546      0.0  Android
2   Italy 2025-08-01         2  0.0800      0.0000      0.0  Android

  - retention: (19, 15)
    Columns: ['country', 'installs', 'retention_rate_d1', 'retention_rate_d2', 'retention_rate_d3', 'retention_rate_d4', 'retention_rate_d5', 'retention_rate_d6', 'retention_rate_d7', 'retention_rate_d10', 'retention_rate_d14', 'retention_rate_d18', 'retention_r